# ODEs: Planetary motion

Here, we simulate planetary motion for the simple 2-body Kepler case, a planar 3-body case, and in a synodic reference frame. 

In [ ]:
import numpy as np
import matplotlib
matplotlib.rcParams['legend.fancybox'] = True
import matplotlib.pyplot as plt
import scipy.integrate
import scipy.optimize


## Kepler motion

In [ ]:
class Kepler:
    '''
    Functor implementing 2-body problem in 2 dimensions
    '''
    def __init__(
            self, 
            G_m1_plus_m2: float, 
            r_aphelion: float, 
            eccentricity: float, 
            step_using_y: bool = False):
        
        self._G_m1_plus_m2 = G_m1_plus_m2
        self._r_aphelion = r_aphelion
        self._eccentricity = eccentricity
        self._a = self._r_aphelion / (1 + self._eccentricity)
        self._vy0 = np.sqrt(self._G_m1_plus_m2 * (2 / self._r_aphelion - 1 / self._a))
        self._step_using_y = step_using_y

    def r_aphelion(self):
        return self._r_aphelion
    
    def vy0(self):
        return self._vy0
        
    def __call__(self, t, p):        
        x = p[0]
        y = p[1]
        vx = p[2]
        vy = p[3]
        r = np.sqrt(x**2 + y**2)
        ax = -self._G_m1_plus_m2 * x / r**3
        ay = -self._G_m1_plus_m2 * y / r**3
        flow = np.array([
            p[2], # dx/dt
            p[3], # dy/dt
            -self._G_m1_plus_m2 * x / r**3, # dvx/dt
            -self._G_m1_plus_m2 * y / r**3, # dvy/dt
        ])
        if self._step_using_y:            # change independent variable from t to y
            flow = flow / vy
        return flow

In [ ]:
kepler = Kepler(G_m1_plus_m2 = 1.0, r_aphelion = 1.0, eccentricity = 0.0)

res = scipy.integrate.solve_ivp(fun=kepler,
                                t_span=(0, 200.),
                                y0=[kepler.r_aphelion(), 0, 0, kepler.vy0()], 
                                vectorized=True,
                                atol=1e-8, rtol=1e-6,
                                method="RK45",
                                first_step = 0.001)
t = res.t
states=res.y
x = states[0]
y = states[1]

In [ ]:
fig = plt.figure(1,figsize=(6,6))
plt.scatter(x,y)
plt.xlabel('x (AU)')
plt.ylabel('y (AU)')
plt.grid()

## Planar 3-body motion

In [ ]:
class Planar3Body :
    '''
    Functor implementing 3-body problem in 2 dimensions
    '''

    def __init__(self, 
                 m1: float, 
                 m2: float,
                 m3: float):
        self._m1 = m1
        self._m2 = m2
        self._m3 = m3
        # use units such that G*(m1+m2+m3) = 4*pi**2
        self._G = 4 * np.pi**2 / (self._m1 + self._m2 + self._m3)

    def __call__(self, t, rv):
        ## This isn't technically necessary, but it makes the code a lot more readable. 
        x1 = rv[0] ; y1 = rv[1]  ;  vx1 = rv[2]  ;  vy1 = rv[3]
        x2 = rv[4] ; y2 = rv[5]  ;  vx2 = rv[6]  ;  vy2 = rv[7]
        x3 = rv[8] ; y3 = rv[9]  ;  vx3 = rv[10] ;  vy3 = rv[11]

        r12 = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
        r13 = np.sqrt((x1 - x3)**2 + (y1 - y3)**2)
        r23 = np.sqrt((x2 - x3)**2 + (y2 - y3)**2)

        ax1 = - self._G * self._m2 * (x1 - x2) / r12**3 - self._G * self._m3 * (x1 - x3) / r13**3
        ay1 = - self._G * self._m2 * (y1 - y2) / r12**3 - self._G * self._m3 * (y1 - y3) / r13**3

        ax2 = - self._G * self._m1 * (x2 - x1) / r12**3 - self._G * self._m3 * (x2 - x3) / r23**3
        ay2 = - self._G * self._m1 * (y2 - y1) / r12**3 - self._G * self._m3 * (y2 - y3) / r23**3

        ax3 = - self._G * self._m1 * (x3 - x1) / r13**3 - self._G * self._m2 * (x3 - x2) / r23**3
        ay3 = - self._G * self._m1 * (y3 - y1) / r13**3 - self._G * self._m2 * (y3 - y2) / r23**3
        return [vx1, vy1, ax1, ay1, vx2, vy2, ax2, ay2, vx3, vy3, ax3, ay3]




In [ ]:
params_planar3body_earthmoonsun = {
    'tau':0.01,
    'accuracy':1e-6,
    'm1':0.012,
    'm2':1.0,
    'm3':330000.0,
    'method':'RK45',
    's1':np.array([0.0, 1.0027, 6.724, 0.0]),      # initial state for mass 1
    's2':np.array([0.0, 1.0,    6.5,   0.0]),      # initial state for mass 2
    's3':np.array([0.0, 0.0,    0,     0.0]),      # initial state for mass 3
    'tmax':1.27       # years
}
planar3body = Planar3Body(m1=0.012, m2=1.0, m3=330000.0)

res = scipy.integrate.solve_ivp(fun=planar3body,
                                t_span=(0, 1.27),
                                y0=np.concatenate((
                                    np.array([0.0, 1.0027, 6.724, 0.0]), 
                                    np.array([0.0, 1.0,    6.5,   0.0]), 
                                    np.array([0.0, 0.0,    0,     0.0]))), 
                                vectorized=True,
                                atol=1e-8,rtol=1e-6,
                                method="RK45",
                                first_step = 0.001)
t = res.t
states=res.y
x1 = states[0]
y1 = states[1]
x2 = states[4]
y2 = states[5]
x3 = states[8]
y3 = states[9]

In [ ]:
fig = plt.figure(2, figsize=(6,6))
plt.scatter(x1,y1, facecolors='none', edgecolors='r', marker="o", s=1)
plt.scatter(x2,y2, facecolors='none', edgecolors='b', marker="o", s=10)
plt.scatter(x3,y3, facecolors='none', edgecolors='k', marker="o", s=20)
plt.show()

## Restricted 3-body motion

In [ ]:
class Rcp3Body:
    '''
    Functor implementing restricted 3-body planar motion
    Following the presentation in the lecture notes, we 
      have only one constant, alpha, via a particular 
      choice of units.
    '''
    # alpha = m2/(m1+m2) in the webnotes
    _alpha = 0.0
    
    # switch to zero in on Poincare section point
    # use y instead of t as independent variable
    _step_using_y = False
    
    def __init__(self, alpha) :
        self._alpha = alpha

        self._eps = 1e-6

        # Lagrange points
        self._L = np.array( [
            [scipy.optimize.bisect(self.f_x, self._alpha - 1 + self._eps, self._alpha - self._eps), 0.0],
            [scipy.optimize.bisect(self.f_x, -1.5, self._alpha - 1 - self._eps), 0.0], 
            [scipy.optimize.bisect(self.f_x, self._alpha + self._eps, 1.5 ), 0.0],
            [self._alpha - 0.5,  np.sqrt(3.0) / 2 ], [self._alpha - 0.5, -np.sqrt(3.0) / 2 ] 
        ]
        )

    def alpha(self):
        return self._alpha
                
    def f_x(self, x):     
        '''
        effective x component of force on the x-axis
        '''
        return (x - (1 - self._alpha) * (x - self._alpha) / np.abs(x - self._alpha) / (x - self._alpha)**2
                   - self._alpha * (x + 1 - self._alpha) / np.abs(x + 1 - self._alpha) / (x + 1 - self._alpha)**2)
        
    def __call__(self, t, rv):         
        '''
        equations in co-rotating frame
        '''
        x = rv[0] ; y = rv[1] ; vx = rv[2] ; vy = rv[3]
        d1 = np.power((x - self._alpha)**2 + y**2, 1.5)
        d2 = np.power((x + 1 - self._alpha)**2 + y**2, 1.5)
        ax = -(1 - self._alpha) * (x - self._alpha) / d1 - self._alpha * (x + 1 -self._alpha) / d2 + x + 2 * vy
        ay = -(1 - self._alpha) * y / d1 - self._alpha * y / d2 + y - 2 * vx
        flow = [vx, vy, ax, ay]
        return flow

In [ ]:
rcp3body = Rcp3Body(alpha=0.25)

res = scipy.integrate.solve_ivp(fun=rcp3body,
                                t_span=(0., 10.),
                                y0=np.array([-0.35, 0.0, 0.0, 0.5]), 
                                vectorized=True,
                                atol=1e-8, rtol=1e-6,
                                method="RK45",
                                first_step = 0.001)
t = res.t
states=res.y
x = states[0]
y = states[1]
vx = states[2]
vy = states[3]



In [ ]:
fig = plt.figure(3, figsize=(6,6))
plt.scatter(rcp3body._L[:,0], rcp3body._L[:,1], color='b', s=10)
plt.scatter(x,y, color='r', s=1)
plt.scatter([rcp3body.alpha()-1], [0], color='g', s=100*(1-rcp3body.alpha()))
plt.scatter([rcp3body.alpha()],   [0], color='g', s=100*rcp3body.alpha())
plt.xlim(-2,2)
plt.ylim(-2,2)
plt.annotate('r1',
            xy=(rcp3body.alpha()-1, 0), xycoords='data',
            xytext=(rcp3body.alpha()-1, 0.1), textcoords='data')
plt.annotate('r2',
            xy=(rcp3body.alpha(), 0), xycoords='data',
            xytext=(rcp3body.alpha(), 0.1), textcoords='data')
plt.annotate('L1',
            xy=rcp3body._L[0], xycoords='data',
            xytext=rcp3body._L[0]+np.array([0.0,0.1]), textcoords='data')
plt.annotate('L2',
            xy=rcp3body._L[1], xycoords='data',
            xytext=rcp3body._L[1]+np.array([0.0,0.1]), textcoords='data')
plt.annotate('L3',
            xy=rcp3body._L[2], xycoords='data',
            xytext=rcp3body._L[2]+np.array([0.0,0.1]), textcoords='data')
plt.annotate('L4',
            xy=rcp3body._L[3], xycoords='data',
            xytext=rcp3body._L[3]+np.array([0.0,0.1]), textcoords='data')
plt.annotate('L5',
            xy=rcp3body._L[4], xycoords='data',
            xytext=rcp3body._L[4]+np.array([0.0,0.1]), textcoords='data')

plt.show()